In [1]:
import numpy as np
import pandas as pd

In [2]:
weather_df = pd.read_csv('data/weather.csv')
rides_df = pd.read_csv('data/cab_rides.csv')

In [3]:
weather_df.describe()

,temp,clouds,pressure,rain,time_stamp,humidity,wind
count,6276.000000,6276.000000,6276.000000,894.000000,6.276000e+03,6276.000000,6276.000000
mean,39.090475,0.677777,1008.445209,0.057652,1.543857e+09,0.763985,6.802812
std,6.022055,0.314284,12.870775,0.100758,6.659340e+05,0.127340,3.633466
min,19.620000,0.000000,988.250000,0.000200,1.543204e+09,0.450000,0.290000
25%,36.077500,0.440000,997.747500,0.004900,1.543387e+09,0.670000,3.517500
50%,40.130000,0.780000,1007.660000,0.014850,1.543514e+09,0.760000,6.570000
75%,42.832500,0.970000,1018.480000,0.060925,1.544691e+09,0.890000,9.920000
max,55.410000,1.000000,1035.120000,0.780700,1.545159e+09,0.990000,18.180000


In [4]:
rides_df.describe()

,distance,time_stamp,price,surge_multiplier
count,693071.000000,6.930710e+05,637976.000000,693071.000000
mean,2.189430,1.544046e+12,16.545125,1.013870
std,1.138937,6.891925e+08,9.324359,0.091641
min,0.020000,1.543204e+12,2.500000,1.000000
25%,1.280000,1.543444e+12,9.000000,1.000000
50%,2.160000,1.543737e+12,13.500000,1.000000
75%,2.920000,1.544828e+12,22.500000,1.000000
max,7.860000,1.545161e+12,97.500000,3.000000


In [5]:
rides_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693071 entries, 0 to 693070
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   distance          693071 non-null  float64
 1   cab_type          693071 non-null  object 
 2   time_stamp        693071 non-null  int64  
 3   destination       693071 non-null  object 
 4   source            693071 non-null  object 
 5   price             637976 non-null  float64
 6   surge_multiplier  693071 non-null  float64
 7   id                693071 non-null  object 
 8   product_id        693071 non-null  object 
 9   name              693071 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 52.9+ MB


In [6]:
rides_df.isna().sum()

distance                0
cab_type                0
time_stamp              0
destination             0
source                  0
price               55095
surge_multiplier        0
id                      0
product_id              0
name                    0
dtype: int64

In [7]:
rides_df.dropna(inplace=True)
#rides_df.isna().sum()

In [8]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6276 entries, 0 to 6275
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   temp        6276 non-null   float64
 1   location    6276 non-null   object 
 2   clouds      6276 non-null   float64
 3   pressure    6276 non-null   float64
 4   rain        894 non-null    float64
 5   time_stamp  6276 non-null   int64  
 6   humidity    6276 non-null   float64
 7   wind        6276 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 392.4+ KB


I assume if there is NaN in rain that means there were no rain.

In [9]:
weather_df['rain'] = weather_df['rain'].fillna(0)

In [10]:
# Converting timestamp to date and removing timestamp column
weather_df['date'] = pd.to_datetime(weather_df['time_stamp'], unit='s')
rides_df['date'] = pd.to_datetime(rides_df['time_stamp'], unit='ms')
weather_df.drop('time_stamp', axis=1, inplace=True)
rides_df.drop('time_stamp', axis=1, inplace=True)

In [11]:
# dealing with error
pd.options.mode.chained_assignment = None 
df = pd.DataFrame()
for location in weather_df['location'].unique():
    # get entries with specific location
    rides_loc = rides_df[rides_df['source'] == location]
    weather_loc = weather_df[weather_df['location'] == location]
    
    rides_loc.sort_values(by='date', inplace=True)
    weather_loc.sort_values(by='date', inplace=True)
    
    # set weather for location from nearest date 
    new_df = weather_loc.set_index('date').reindex(rides_loc.set_index('date').index, method='nearest').reset_index()
    final_df = pd.merge(rides_loc, new_df, on='date')
    
    final_df.drop_duplicates(inplace=True)
    # adding to new, empty DF
    df = pd.concat([df, final_df])

In [12]:
df['surge_multiplier'].value_counts()

1.00    617001
1.25     11085
1.50      5065
1.75      2420
2.00      2239
2.50       154
3.00        12
Name: surge_multiplier, dtype: int64

Now let's remove some usless columns for example:
 - id column
 - product_id (similar to name)
 - pressure, wind and  humidity (let's just limit ourselves to rain (+clouds) and temperature
 - surge_multiplier only a few entries are different from 1.0
 - date, all data comes from only 2 weeks so its really short period of time
 

In [13]:
df.drop(['id', 'product_id', 'pressure', 'wind', 'humidity', 'surge_multiplier', 'location'], axis=1, inplace=True)

In [14]:
df.head()

,distance,cab_type,destination,source,price,name,date,temp,clouds,rain
0,6.26,Uber,South Station,Back Bay,34.0,Black,2018-11-26 03:40:46.971,41.04,0.87,0.0
1,2.99,Lyft,North End,Back Bay,19.5,Lyft XL,2018-11-26 03:40:47.022,41.04,0.87,0.0
2,1.66,Lyft,Fenway,Back Bay,26.0,Lux,2018-11-26 03:40:47.023,41.04,0.87,0.0
3,2.31,Lyft,Haymarket Square,Back Bay,16.5,Lux,2018-11-26 03:40:47.738,41.04,0.87,0.0
4,2.31,Lyft,Haymarket Square,Back Bay,16.5,Lyft XL,2018-11-26 03:40:47.747,41.04,0.87,0.0


In [15]:
# Adding new column, to compare prices with day of week
df['day_of_week'] = df['date'].dt.day_of_week
df['day_of_week'] = df['day_of_week'].astype('category')


In [16]:
# change some columns to categorical values
df['cab_type'] = df['cab_type'].astype('category')
df['destination'] = df['destination'].astype('category')
df['source'] = df['source'].astype('category')
df['name'] = df['name'].astype('category')

Now we can remove date column because all data comes from short period of time (~2 weeks) so it should not affect fares

In [19]:
df.drop('date',axis=1,  inplace=True)
df.head()

KeyError: "['date'] not found in axis"

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637976 entries, 0 to 178931
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   distance     637976 non-null  float64 
 1   cab_type     637976 non-null  category
 2   destination  637976 non-null  category
 3   source       637976 non-null  category
 4   price        637976 non-null  float64 
 5   name         637976 non-null  category
 6   temp         637976 non-null  float64 
 7   clouds       637976 non-null  float64 
 8   rain         637976 non-null  float64 
 9   day_of_week  637976 non-null  category
dtypes: category(5), float64(5)
memory usage: 32.2 MB


# Visualization
Some plots to explore if we have a balanced dataset and wchich feature affects price the most

In [ ]:
import seaborn as sns
import matplot